<h2>Week - 3 Assignment </h2>

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<hr></hr>
<h4> As Per the requirements
<pre><I>"More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, 
you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
These two rows will be combined into one row with the neighborhoods separated with a comma as shown 
in row 11 in the above table"</I></pre></h4>
        
<h4><pre>Get the postal code data from the Web url and then group on the PostalCode column. 
    Then combine the same PostalCode Neighborhoods into 
    a single row where neighborhoods are comma separated for each PostalCode.</pre></h4>
<hr></hr>

In [6]:
##Extract and Prase the Postal code data from the URL 
from bs4 import BeautifulSoup
!wget -q -O 'toronto_data.htm' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
f = open("toronto_data.htm", "r")
html = f.read()
soup = BeautifulSoup(html,features="html.parser")
data = [
    [
        [ str(td.text).strip() for td in tr.find_all('td') if td.text ]
        for tr in table.find_all('tr')[1:]
    ]
    for table in soup.find_all('table', class_="wikitable")
    ]

# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
# instantiate the dataframe
neighborhoodsT = pd.DataFrame(columns=column_names)
##Create dataframe from Postal table data
df = pd.DataFrame(data[0],columns=['PostalCode', 'Borough', 'Neighborhood'])
df1 = df[df['Borough'] != "Not assigned"].reset_index(drop=True);
df1['Neighborhood'] = df1.apply(lambda x: x['Borough'] if x['Neighborhood']=="Not assigned" else x['Neighborhood'] , axis=1)
df1.groupby("PostalCode")

pin_code=""
prev_pin_code=""
combined_nh=""
prev_nh=""
cnt=0
idx=0;
for row in df1.itertuples():
    #print(br, prev_br)
    pin_code = df1.at[row.Index, "PostalCode"];
    if(prev_pin_code==""):
        prev_pin_code=pin_code;
    if(pin_code != prev_pin_code):
        prev_pin_code=pin_code;
        df1.loc[row.Index-1,"Neighborhood"] = combined_nh
        neighborhoodsT = neighborhoodsT.append(df1.loc[row.Index-1])
        idx=idx+1;
        combined_nh = df1.at[row.Index, "Neighborhood"]
        cnt=0
    else:
        cnt=cnt+1;
        if(cnt >= 1 and combined_nh!=""):
            combined_nh += ","      
        combined_nh += df1.at[row.Index, "Neighborhood"]
        
#print(df1.head(100))   
neighborhoodsT.head(50)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Harbourfront,Regent Park"
5,M6A,North York,"Lawrence Heights,Lawrence Manor"
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Rouge,Malvern"
10,M3B,North York,Don Mills North
12,M4B,East York,"Woodbine Gardens,Parkview Hill"
14,M5B,Downtown Toronto,"Ryerson,Garden District"


In [7]:
neighborhoodsT.shape

(102, 3)